In [1]:
# Import packages
import numpy as np
import pandas as pd
import chardet
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import string
import time

# Display plots inline
# % matplotlib inline

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize


from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.preprocessing import LabelBinarizer
from sklearn.svm import SVC

from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, Embedding
from keras.layers.convolutional import Conv1D, MaxPooling1D
# from keras.layers.merge import concatenate
from tensorflow.keras.layers import concatenate, Add, Subtract
from keras.optimizers import Adam
from keras.preprocessing.text import one_hot
from keras.callbacks import ModelCheckpoint 

from scipy import sparse, stats
import random
np.random.seed(42)
random.seed(42)

In [2]:
import chardet
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

with open(r"c:\Users\guyu8\Documents\WeChat Files\wxid_8m1bo9kcmqmy21\FileStorage\File\2024-05\data_with_genre.csv", 'rb') as file:
    result = chardet.detect(file.read())
    encoding = result['encoding']

data = pd.read_csv(r"c:\Users\guyu8\Documents\WeChat Files\wxid_8m1bo9kcmqmy21\FileStorage\File\2024-05\data_with_genre.csv", encoding=encoding)
text = list(data['text'].values)
author = list(data['author'].values)

normed_text = []
for excerpt in text:
    excerpt = excerpt.replace('\xa0', '')
    excerpt = excerpt.lower().replace("'", " ")
    excerpt = excerpt.translate(str.maketrans('', '', string.punctuation))
    excerpt = excerpt.replace('“', '').replace('”', '')
    normed_text.append(excerpt)

tokens = [word_tokenize(sentence) for sentence in normed_text]
flat_tokens = [token for sublist in tokens for token in sublist]

unique_words = set(flat_tokens)
vocab = {word: idx for idx, word in enumerate(sorted(unique_words))}

train_data, test_data, author_train, author_test = train_test_split(normed_text, author, test_size=0.2, random_state=5)
train_tokens = [word_tokenize(sentence) for sentence in train_data]
# train_tokens = [element for sublist in train_tokens for element in sublist]

test_tokens = [word_tokenize(sentence) for sentence in test_data]
normed_text = [word_tokenize(sentence) for sentence in normed_text]

# train_data = [[vocab[word] for word in word_tokenize(text)] for text in train_data]
# test_data = [[vocab[word] for word in word_tokenize(text)] for text in test_data]

In [3]:
def create_n_grams(sentence_list, n, vocab_size, seq_size):
    n_gram_list = []
    for sentence in sentence_list:
        n_grams = [sentence[i:i+n] for i in range(len(sentence) - n + 1)]
        combined_n_grams = [" ".join(sublist) for sublist in n_grams]
        new_string = ["".join(word.split()) for word in combined_n_grams]
        new_string = " ".join(new_string)
        hot = one_hot(new_string, round(vocab_size*1.3))

        hot_len = len(hot)
        if hot_len >= seq_size:
            hot = hot[0:seq_size]
        else:
            diff = seq_size - hot_len
            extra = [0]*diff
            hot = hot + extra

        n_gram_list.append(hot)
        
    n_gram_array = np.array(n_gram_list)
    
    return n_gram_array

In [4]:
def get_vocab_size(sentence_list, n, seq_size):

    token_vocab = []

    for sentence in sentence_list:
        n_grams = [sentence[i:i+n] for i in range(len(sentence) - n + 1)]
        combined_three_grams = [" ".join(sublist) for sublist in n_grams]
        new_string = ["".join(word.split()) for word in combined_three_grams]
        token_vocab.append(new_string)
    flattened_list = [item for sublist in token_vocab for item in sublist]

    n_gram_cnt = Counter(flattened_list)
    vocab_size = len(n_gram_cnt)
    
    
    return vocab_size

In [5]:
gram1_train = create_n_grams(train_tokens, 1, get_vocab_size(train_tokens, 1, 300), 300)
gram1_test = create_n_grams(test_tokens, 1, get_vocab_size(test_tokens, 1, 300), 300)
gram1 = create_n_grams(normed_text, 1, get_vocab_size(normed_text, 1, 300), 300)
max_1gram = np.max(gram1_train)
max_1gram = np.max(gram1)

In [121]:
# author_lb = LabelBinarizer()
# author_lb.fit(author_train)
# author_train_hot = author_lb.transform(author_train)
# author_test_hot = author_lb.transform(author_test)

author_lb = LabelBinarizer()
author_lb.fit(author)
author_hot = author_lb.transform(author)
author_test_hot = author_lb.transform(author_test)
author_train_hot = author_lb.transform(author_train)

In [130]:
print(author[1])
print(author_hot[1])

Leo Tolstoy
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]


In [134]:
print(author_test[4])

Leo Tolstoy


In [137]:
print(np.argmax(author_test_hot[4]))

13


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class AdaptiveDropout(Layer):
    def __init__(self, initial_rate=0.2):
        super(AdaptiveDropout, self).__init__()
        self.dropout_rate = tf.Variable(initial_rate, trainable=False)

    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.dropout_rate)
        return inputs

    def set_rate(self, new_rate):
        self.dropout_rate.assign(new_rate)

class AdjustDropoutRateCallback(tf.keras.callbacks.Callback):
    def __init__(self, model, threshold=0.90, increase_by=0.1):
        super().__init__()
        for layer in model.layers:
            if isinstance(layer, AdaptiveDropout):
                layer_adap = layer
                break
        self.layer = layer_adap
        self.threshold = threshold
        self.increase_by = increase_by

    def on_epoch_end(self, epoch, logs=None):
        if logs['accuracy'] > self.threshold:
            new_rate = min(1.0, self.layer.dropout_rate + self.increase_by)
            self.layer.set_rate(new_rate)
            print(f"Epoch {epoch+1}: Increasing dropout rate to {new_rate:.2f}")
        else:
            print(f"Epoch {epoch+1}: Current accuracy {logs['accuracy']:.2f} is below threshold")


In [8]:
from tensorflow.keras.layers import Input, Embedding, Dropout, Conv1D, MaxPooling1D, Flatten, Dense, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def define_model2(input_len, output_size, vocab_size, embedding_dim, verbose=True,
                  drop_out_pct=0.25, conv_filters=500, activation_fn='leaky_relu',
                  pool_size=2, learning=0.001, init_mode_embedding=None, init_mode_conv=None, channel = 4):
    """Define n-gram CNN with different weight initializations for each channel.
    
    Args:
    input_len: int. Length of input sequences.
    output_size: int. Number of output classes.
    vocab_size: int. Maximum value of n-gram encoding.
    embedding_dim: int. Size of embedding layer.
    verbose: bool. Whether or not to print model summary.
    drop_out_pct: float. Drop-out rate.
    conv_filters: int. Number of filters in the conv layer.
    activation_fn: string. Activation function to use in the convolutional layer.
    pool_size: int. Pool size for the max pooling layer.
    learning: float. Learning rate for the model optimizer.
    init_mode_embedding: list of keras.initializers. Initializers for the embedding layers.
    init_mode_conv: list of keras.initializers. Initializers for the convolutional layers.
    
    Returns:
    model: keras model object. 
    """
    if init_mode_embedding is None:
        init_mode_embedding = [None] * channel
    if init_mode_conv is None:
        init_mode_conv = [None] * channel

    channels = []
    inputs = []

    for i in range(channel):
        input_shape = Input(shape=(input_len,))
        inputs.append(input_shape)
        embedding = Embedding(vocab_size, embedding_dim, embeddings_initializer=init_mode_embedding[i])(input_shape)
        drop = AdaptiveDropout(drop_out_pct)(embedding)
        conv = Conv1D(filters=conv_filters, kernel_size=i+3, activation=activation_fn, kernel_initializer=init_mode_conv[i])(drop)
        pool = MaxPooling1D(pool_size=pool_size)(conv)
        flat = Flatten()(pool)
        channels.append(flat)

    # Merge channels
    merged = concatenate(channels)
    
    
    output = Dense(output_size, activation='softmax')(merged)
   
    model = Model(inputs=inputs, outputs=output)
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning), metrics=['accuracy'])
    
    if verbose:
        model.summary()
        
    return model


In [61]:
import numpy as np
from tensorflow.keras.utils import to_categorical

def train_models(X_train, y_train, num_models=3, num_channels = 4):
    models = []
    n_samples = X_train.shape[0]
    output_size = len(np.unique(y_train))

    for _ in range(num_models):
        indices = np.random.choice(n_samples, n_samples, replace=True)
        X_sample = X_train[indices]
        y_sample = y_train[indices]

        model =  define_model2(300, 23, max_1gram + 1, 16, learning=0.1, drop_out_pct=0.0001, channel=num_channels)
        model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
        dropout_adjustment_callback = AdjustDropoutRateCallback(model)
        model.fit([X_sample]*num_channels, y_sample, epochs=2, batch_size=16, 
                verbose = 1, validation_split = 0.2, callbacks=[dropout_adjustment_callback])
        models.append(model)
    
    return models


def majority_vote(models, X_test, num_channels = 4):

    predictions = np.array([model.predict([X_test]*num_channels) for model in models])
    # print(predictions)

    predictions = np.sum(predictions, axis=0)

    predictions = predictions/len(models)



    return predictions


In [10]:
gram1_model2 = define_model2(300, 23, max_1gram + 1, 16, learning=0.1, drop_out_pct=0.0001)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 300)]        0           []                               
                                                                                              

In [63]:
models = train_models(X_train=gram1, y_train=author_hot, num_models=11)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 300)]        0           []                               
                                                                                            

Epoch 1/2
658/658 [==============================] - 91s 135ms/step - loss: 0.9413 - accuracy: 0.7150 - val_loss: 0.2441 - val_accuracy: 0.9289
Epoch 2/2
658/658 [==============================] - 123s 187ms/step - loss: 0.0278 - accuracy: 0.9928 - val_loss: 0.1799 - val_accuracy: 0.9483


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 input_10 (InputLayer)          [(None, 300)]        0           []                               
                                                                                                  
 input_11 (InputLayer)          [(None, 300)]        0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, 300)]        0           []                               
                                                                                            

Epoch 1/2
658/658 [==============================] - 136s 204ms/step - loss: 0.9215 - accuracy: 0.7193 - val_loss: 0.2013 - val_accuracy: 0.9342
Epoch 2/2
658/658 [==============================] - 125s 189ms/step - loss: 0.0249 - accuracy: 0.9932 - val_loss: 0.1629 - val_accuracy: 0.9510


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 300)]        0           []                               
                                                                                                  
 input_14 (InputLayer)          [(None, 300)]        0           []                               
                                                                                                  
 input_15 (InputLayer)          [(None, 300)]        0           []                               
                                                                                                  
 input_16 (InputLayer)          [(None, 300)]        0           []                               
                                                                                            

...

Epoch 1/2
658/658 [==============================] - 124s 186ms/step - loss: 0.9162 - accuracy: 0.7196 - val_loss: 0.2027 - val_accuracy: 0.9411
Epoch 2/2
658/658 [==============================] - 121s 184ms/step - loss: 0.0240 - accuracy: 0.9933 - val_loss: 0.2693 - val_accuracy: 0.9244


Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_41 (InputLayer)          [(None, 300)]        0           []                               
                                                                                                  
 input_42 (InputLayer)          [(None, 300)]        0           []                               
                                                                                                  
 input_43 (InputLayer)          [(None, 300)]        0           []                               
                                                                                                  
 input_44 (InputLayer)          [(None, 300)]        0           []                               
                                                                                           

Epoch 1/2
658/658 [==============================] - 122s 182ms/step - loss: 0.9688 - accuracy: 0.7036 - val_loss: 0.2213 - val_accuracy: 0.9304
Epoch 2/2
658/658 [==============================] - 133s 202ms/step - loss: 0.0295 - accuracy: 0.9906 - val_loss: 0.1728 - val_accuracy: 0.9498


Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_45 (InputLayer)          [(None, 300)]        0           []                               
                                                                                                  
 input_46 (InputLayer)          [(None, 300)]        0           []                               
                                                                                                  
 input_47 (InputLayer)          [(None, 300)]        0           []                               
                                                                                                  
 input_48 (InputLayer)          [(None, 300)]        0           []                               
                                                                                           

Epoch 1/2
658/658 [==============================] - 195s 289ms/step - loss: 0.9159 - accuracy: 0.7200 - val_loss: 0.2196 - val_accuracy: 0.9396
Epoch 2/2
658/658 [==============================] - 176s 267ms/step - loss: 0.0331 - accuracy: 0.9904 - val_loss: 0.2061 - val_accuracy: 0.9437


In [153]:
predictions = majority_vote(models, gram1)

412/412 [==============================] - 24s 58ms/step


In [154]:
true = np.argmax(author_hot, axis = 1)

accuracy = tf.metrics.Accuracy()

accuracy.update_state(true, np.argmax(predictions, axis = 1))

print("Accuracy:", accuracy.result().numpy())

Accuracy: 0.99908775
